In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

API_TOKEN = "KuQ4Rt1ypOCvXfcm3cZXdPhOUlbuOrpHBgJkFm1MWTvtRR8TLhgEI02hjDxz"
BASE_URL = "https://api.sportmonks.com/v3/football"


# -------------------------
# Fixtures (next 12 hours)
# -------------------------
def fetch_fixtures_next_12h() -> pd.DataFrame:
    now_utc = datetime.now(timezone.utc)
    end_utc = now_utc + timedelta(hours=12)

    start_date = now_utc.date()
    end_date = (now_utc + timedelta(days=1)).date()
    endpoint = f"/fixtures/between/{start_date:%Y-%m-%d}/{end_date:%Y-%m-%d}"

    session = requests.Session()
    fixtures = []
    page = 1

    while True:
        r = session.get(
            f"{BASE_URL}{endpoint}",
            params={"api_token": API_TOKEN, "page": page},
            timeout=30,
        )
        r.raise_for_status()
        payload = r.json()

        fixtures.extend(payload.get("data") or [])

        pagination = ((payload.get("meta") or {}).get("pagination")) or {}
        if pagination.get("current_page") is None or pagination.get("current_page") >= pagination.get("total_pages", 0):
            break
        page += 1

    df = pd.DataFrame(
        {
            "FixtureID": [f.get("id") for f in fixtures],
            "Fixture": [f.get("name") for f in fixtures],
            "StartingAt": [f.get("starting_at") for f in fixtures],
        }
    )

    if df.empty:
        return df

    df["StartingAt"] = pd.to_datetime(df["StartingAt"], utc=True, errors="coerce")
    df = df[df["StartingAt"].between(now_utc, end_utc)].reset_index(drop=True)
    return df


# -------------------------
# Odds (schema-faithful)
# -------------------------
def fetch_odds_prematch_for_fixture(fixture_id: int, session: requests.Session) -> list[dict]:
    endpoint = f"/odds/pre-match/fixtures/{fixture_id}"

    r = session.get(
        f"{BASE_URL}{endpoint}",
        params={"api_token": API_TOKEN},
        timeout=30,
    )
    if r.status_code == 404:
        return []

    r.raise_for_status()
    odds = r.json().get("data") or []

    rows = []
    for odd in odds:
        rows.append(
            {
                # Core identifiers
                "OddID": odd.get("id"),
                "FixtureID": odd.get("fixture_id"),
                "MarketID": odd.get("market_id"),
                "BookmakerID": odd.get("bookmaker_id"),

                # Descriptions
                "MarketDescription": odd.get("market_description"),
                "Label": odd.get("label"),
                "Name": odd.get("name"),

                # Prices
                "Value": odd.get("value"),
                "Probability": odd.get("probability"),
                "DP3": odd.get("dp3"),
                "Fractional": odd.get("fractional"),
                "American": odd.get("american"),

                # Result state
                "Winning": odd.get("winning"),
                "Stopped": odd.get("stopped"),

                # Line information
                "Total": odd.get("total"),
                "Handicap": odd.get("handicap"),

                # Player / participants
                "Participants": odd.get("participants"),

                # Metadata
                "LatestBookmakerUpdate": odd.get("latest_bookmaker_update"),
            }
        )

    return rows


# -------------------------
# Combined fetch
# -------------------------
def fetch_odds_next_12h() -> pd.DataFrame:
    fixtures_df = fetch_fixtures_next_12h()
    if fixtures_df.empty:
        return pd.DataFrame()

    session = requests.Session()
    rows = []

    for fid in fixtures_df["FixtureID"].astype(int):
        rows.extend(fetch_odds_prematch_for_fixture(fid, session))

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    # Join fixture metadata
    df = df.merge(fixtures_df, on="FixtureID", how="left")

    # ---- Type normalization (non-destructive) ----
    df["Odds"] = pd.to_numeric(df["Value"], errors="coerce")
    df["Total"] = pd.to_numeric(df["Total"], errors="coerce")
    df["Handicap"] = pd.to_numeric(df["Handicap"], errors="coerce")

    df["Probability"] = pd.to_numeric(df["Probability"], errors="coerce")
    df["DP3"] = pd.to_numeric(df["DP3"], errors="coerce")

    # Keep original string formats too (fractional / american untouched)

    # Drop unusable odds
    df = df.dropna(subset=["Odds", "MarketID", "BookmakerID", "Label", "StartingAt"]).reset_index(drop=True)

    return df


# -------------------------
# Example usage
# -------------------------
df = fetch_odds_next_12h()

df[
    [
        "Fixture",
        "StartingAt",
        "MarketDescription",
        "Label",
        "Odds",
        "Total",
        "Handicap",
        "Participants",
        "BookmakerID",
        "Probability",
        "Fractional",
        "American",
        "Winning",
        "Stopped",
        "LatestBookmakerUpdate",
    ]
].head(50)

,Fixture,StartingAt,MarketDescription,Label,Odds,Total,Handicap,Participants,BookmakerID,Probability,Fractional,American,Winning,Stopped,LatestBookmakerUpdate
0,Lecce vs Roma,2026-01-06 17:00:00+00:00,Team Shots,1,1.83,NaN,NaN,None,2,NaN,11/6,-121,False,True,2026-01-04 19:40:37
1,Lecce vs Roma,2026-01-06 17:00:00+00:00,Team Shots,1,1.83,NaN,NaN,None,2,NaN,11/6,-121,False,True,2026-01-04 19:40:37
2,Lecce vs Roma,2026-01-06 17:00:00+00:00,1st Half Goal Line,Under,1.72,1.0,NaN,None,2,NaN,50/29,-138,False,False,2026-01-06 14:21:37
3,Lecce vs Roma,2026-01-06 17:00:00+00:00,Alternative 1st Half Goal Line,Under,1.67,1.0,NaN,None,2,NaN,57/34,-149,False,True,2026-01-06 13:40:44
4,Lecce vs Roma,2026-01-06 17:00:00+00:00,Player Shots,4.5,11.00,NaN,NaN,None,2,NaN,11,1000,False,False,2026-01-06 14:21:37
5,Lecce vs Roma,2026-01-06 17:00:00+00:00,Player Shots,4.5,21.00,NaN,NaN,None,2,NaN,21,2000,False,False,2026-01-06 14:21:37
6,Lecce vs Roma,2026-01-06 17:00:00+00:00,Goalscorers,Anytime,8.50,NaN,NaN,None,2,NaN,17/2,750,False,False,2026-01-06 14:21:37
7,Lecce vs Roma,2026-01-06 17:00:00+00:00,Goalscorers,First,34.00,NaN,NaN,None,2,NaN,34,3300,False,False,2026-01-06 14:21:37
8,Lecce vs Roma,2026-01-06 17:00:00+00:00,Goalscorers,First,41.00,NaN,NaN,None,2,NaN,41,4000,False,False,2026-01-06 14:21:37
9,Lecce vs Roma,2026-01-06 17:00:00+00:00,Goalscorers,Last,29.00,NaN,NaN,None,2,NaN,29,2800,False,False,2026-01-06 14:21:37
